In [ ]:
from bs4 import BeautifulSoup                                       # HTML və XML-dən məlumat çıxarmaq üçün
from selenium import webdriver                                      # Brauzeri avtomatik idarə etmək üçün
from selenium.webdriver.chrome.service import Service               # ChromeDriver-i başlatmaq üçün
from selenium.webdriver.common.by import By                         # HTML elementlərini axtarmaq üçün (məs: By.CLASS_NAME)
from webdriver_manager.chrome import ChromeDriverManager            # ChromeDriver versiyasını avtomatik endirmək üçün
from selenium.webdriver.chrome.options import Options               # Chrome brauzerinə xüsusi ayarlar vermək üçün
from selenium.webdriver.support.ui import WebDriverWait             # Elementlərin yüklənməsini gözləmək üçün
from selenium.webdriver.support import expected_conditions as EC    # Müəyyən şərtlər ödənənə qədər gözləmək üçün
from selenium.common.exceptions import TimeoutException             # Vaxt aşımı xətasını idarə etmək üçün
import time                                                         # Kodda gecikmə vermək üçün (məs: time.sleep)
import pandas as pd                                                 # Verilənlərlə işləmək üçün (DataFrame, CSV və s.)
import re                                                           # Mətn təmizləmək, regex (müntəzəm ifadələr) ilə işləmək üçün
import pyodbc                                                       # SQL Server (və digər ODBC dəstəkləyən verilənlər bazaları) ilə bağlantı qurmaq üçün
from sqlalchemy import create_engine                                # SQLAlchemy ilə verilənlər bazasına bağlantı yaratmaq və pandas ilə işləmək üçün
import requests
import os
from selenium.common.exceptions import NoSuchElementException
from bs4 import NavigableString
import datetime
import json
import queue
from threading import Thread
import threading
from concurrent.futures import ThreadPoolExecutor
import warnings
warnings.filterwarnings('ignore')
import numpy as np


#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


raw_data = queue.Queue()
cleaned_data = queue.Queue()
list_data = []
see_link = set()  
data_lock = threading.Lock()  


# "https://emlak.az/elanlar?ann_type=3"

def scrap():    
    base_urls = [
                 "https://emlak.az/elanlar?ann_type=3&search_type_check=1&property_type=1&room_min=&room_max=&space_min=&space_max=&price_min=&price_max=",
                 "https://emlak.az/elanlar?ann_type=3&search_type_check=1&property_type=2&room_min=&room_max=&space_min=&space_max=&price_min=&price_max=",
                 "https://emlak.az/elanlar?ann_type=3&search_type_check=1&property_type=3&room_min=&room_max=&space_min=&space_max=&price_min=&price_max=",
                 "https://emlak.az/elanlar?ann_type=3&search_type_check=1&property_type=4&room_min=&room_max=&space_min=&space_max=&price_min=&price_max=",
                 "https://emlak.az/elanlar?ann_type=3&search_type_check=1&property_type=5&room_min=&room_max=&space_min=&space_max=&price_min=&price_max=",
                 "https://emlak.az/elanlar?ann_type=3&search_type_check=1&property_type=6&room_min=&room_max=&space_min=&space_max=&price_min=&price_max=",
                 "https://emlak.az/elanlar?ann_type=3&search_type_check=1&property_type=7&room_min=&room_max=&space_min=&space_max=&price_min=&price_max=",
                 "https://emlak.az/elanlar?ann_type=3&search_type_check=1&property_type=8&room_min=&room_max=&space_min=&space_max=&price_min=&price_max=",
                 "https://emlak.az/elanlar?ann_type=3&search_type_check=1&property_type=10&room_min=&room_max=&space_min=&space_max=&price_min=&price_max="]


    def scrape_url(current_url):
        global list_data, raw_data, see_link, data_lock
        options = webdriver.ChromeOptions()
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-gpu')
        # options.add_argument('--headless')  # İstəsəniz açıb headless edə bilərsiniz

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        # driver.maximize_window()

        base = current_url.split("&page=")[0]
        for i in range(1, 1001):
            url = f"{base}&page={i}" if "?" in base else f"{base}?page={i}"


            #--------------------------------------------------------------------------------------------------------
            for attempt in range(3):  
                try:
                    driver.set_page_load_timeout(45)
                    driver.get(url)
                    break  
                except TimeoutException:
                    print(f"{i}. səhifə - Cəhd {attempt+1}: vaxt limiti keçdi")
                    time.sleep(5)

            try:
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.ticket-list > div > div.img > a")))
            except TimeoutException:
                print(f"{i}. səhifə yüklənmədi, keçilir")
                continue

            time.sleep(1)

            #--------------------------------------------------------------------------
            # Elan linklərinin tapılması
            elements = driver.find_elements(By.CSS_SELECTOR, "div.ticket-list > div > div.img > a")
            links = [e.get_attribute("href") for e in elements if e.get_attribute("href")]

            # Tapılan elan linklərindən məlumatların toplanması
            new_links_count = 0
            for link in links:
                # Thread-safe şəkildə link yoxla və əlavə et
                with data_lock:
                    if link in see_link:
                        continue  # Link daha əvvəl görülübsə, keç
                    see_link.add(link)  # Görülmüş linklərə əlavə et
                
                new_links_count += 1
                
                #--------------------------------------------------------------------------
                # səhifənin yüklənməsi zamanı baş verə biləcək TimeoutException halları üçün 2 defə cəhd etmək
                for attempt in range(3):  
                    try:
                        driver.set_page_load_timeout(45)
                        driver.get(link)
                        break
                    except TimeoutException:
                        print(f"Elan səhifəsi {link} - Cəhd {attempt+1}: vaxt limiti keçdi")
                        time.sleep(5)
                
                #--------------------------------------------------------------------------
                # Elanın məlumatlarının yüklənməsini gözləyir
                try:
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "h1")))
                except TimeoutException:
                    print(f"Elan yüklənmədi: {link}")
                    continue        
                

                #--------------------------------------------------------------------------
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                data = {"URL": link}

                #--------------------------------------------------------------------------
                # Elan id
                ann_id = soup.find("p", {"class":"pull-right"})
                data["ListingID"] = ann_id.text.strip() if ann_id else None

                #--------------------------------------------------------------------------
                # Elan adi 
                announce = soup.find("h1", {"class":"title"})
                data["ListingName"] = announce.text.strip() if announce else None

                #--------------------------------------------------------------------------
                # Qiymeti 
                price = soup.find("span", {"class":"m"})
                data["Price"] = price.text.strip() if price else None

                #--------------------------------------------------------------------------
                # Paylasilma tarixi
                date = soup.find("span", {"class":"date"})
                data["Date"] = date.text.strip() if date else None

                #--------------------------------------------------------------------------
                # Haqqinda 
                description = soup.find("div", {"class":"desc"})
                data["Description"] = description.text.strip() if description else None

                #--------------------------------------------------------------------------
                # Əvvəlcədən boş sahələr
                data["CategoryID"] = None
                data["FloorNumber"] = None
                data["FieldAreaSqm"] = None
                data["CountRoom"] = None
                data["Title"] = None
                data["Mortgage"] = None
                data["Repaired"] = None

                # Mapping: saytın label-ları -> bizim column adlarımız
                label_mapping = {
                    "Əmlakın növü": "CategoryID",
                    "Mərtəbə sayı": "FloorNumber",
                    "Sahə": "FieldAreaSqm",
                    "Otaqların sayı": "CountRoom",
                    "Sənədin tipi": "Title",
                    "İpoteka": "Mortgage",
                    "Təmiri": "Repaired",
                }

                # Elan səhifəsindəki label-ları tapırıq
                labels = soup.find_all("span", class_="label")
                for label in labels:
                    key = label.get_text(strip=True)

                    # Əgər label bizim mapping-dədirsə, yəni lazım olan məlumatdırsa
                    if key in label_mapping:
                        mapped_key = label_mapping[key]

                        # Text ya birbaşa sibling kimi, ya da bir tag içində ola bilər
                        next_el = label.find_next_sibling(string=True)
                        if next_el and next_el.strip():
                            value = next_el.strip()
                        else:
                            next_tag = label.find_next_sibling()
                            value = next_tag.get_text(strip=True) if next_tag else None

                        data[mapped_key] = value

                #--------------------------------------------------------------------------
                # Unvan 
                location = soup.find("div", {"class":"map-address"})
                data["Address"] = location.text.strip() if location else None

                #--------------------------------------------------------------------------
                # Koordinatlar
                value = soup.find("input", id="google_map")
                if value:
                    lat, lon = value["value"].strip("()").split(",")
                    data["Latitude"] = lat.strip()
                    data["Longitude"] = lon.strip()
                else:
                    data["Latitude"] = None
                    data["Longitude"] = None

                #--------------------------------------------------------------------------
                # Satici 
                owner = soup.find("p", {"class":"name-seller"})
                data["SalesPerson"] = owner.text.strip() if owner else None

                #--------------------------------------------------------------------------
                # Satici novu
                seller_info = soup.find("p", {"class":"name-seller"})
                seller_type_span = seller_info.find("span") if seller_info else None
                data["SellerType"] = seller_type_span.text.strip() if seller_type_span else None

                #--------------------------------------------------------------------------
                # Nomre
                contact = soup.find("p", {"class":"phone"})
                data["Contact"] = contact.text.strip() if contact else None


                #--------------------------------------------------------------------------
                # Şəkilləri topla
                image_links = []
                image_tags = soup.find_all("img")
                for img in image_tags:
                    src = img.get("src") or img.get("data-src")
                    if src and "/images/announces/" in src:
                        if src.startswith("/"):
                            src = "https://emlak.az" + src
                        image_links.append(src)
                data["Images"] = image_links


                #--------------------------------------------------------------------------
                list_data.append(data)

                df = pd.DataFrame([data])
                raw_data.put(df)

                #--------------------------------------------------------------------------
                print(f"URL: {current_url} - Səhifə {i} - Ümumi {len(list_data)} elan toplandı.")

        driver.quit()
        return list_data
    
    with ThreadPoolExecutor(max_workers=len(base_urls)) as executor:
        executor.map(scrape_url, base_urls)

#----------------------------------------------------------------------------------------------------------------------------------------
def preprocess(raw):
    df = raw.copy()
    
    # URL - string formatına keçir
    if 'URL' in df.columns:
        df['URL'] = df['URL'].astype(str)

    # ListingID - "Elanın kodu:" hissəsini silir və int64-ə keçir
    if 'ListingID' in df.columns:
        df['ListingID'] = df['ListingID'].astype(str).str.replace('Elanın kodu:', '', regex=True)
        df['ListingID'] = pd.to_numeric(df['ListingID'], errors='coerce')
        df['ListingID'] = df['ListingID'].apply(lambda x: '303' + str(int(x)) if pd.notna(x) else x)
        df['ListingID'] = pd.to_numeric(df['ListingID'], errors='coerce').astype("Int64")


    # ListingName - string olur
    if 'ListingName' in df.columns:
        df['ListingName'] = df['ListingName'].astype(str)


    # SalesType sütununu təyin edirik
    df['SalesType'] = df['ListingName'].apply(lambda x: 0 if 'İcarəyə verilir' in x else (1 if 'Satılır' in x else None))
    df['SalesType'] = df['SalesType'].astype(bool)


    # Price - int64 olur
    if 'Price' in df.columns:
        df['Price'] = df['Price'].astype(str).str.replace(r'[^\d]', '', regex=True)
        df['Price'] = pd.to_numeric(df['Price'], errors='coerce').astype('Int64')


    df['DailyRent'] = df.apply(lambda row: None if row['SalesType'] == True else (True if pd.notna(row['Price']) and row['Price'] < 1000 else False), axis=1)
    df['DailyRent'] = df['DailyRent'].astype('boolean')



    # Date - "Elanın tarixi:" hissəsini silir və düzgün formata keçir
    if 'Date' in df.columns:
        df['Date'] = df['Date'].astype(str).str.replace('Elanın tarixi:', '', regex=False).str.strip()
        df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y', errors='coerce')

    # Description - emoji və xüsusi simvolları silir, \n-ləri silir
    if 'Description' in df.columns:
        df['Description'] = df['Description'].astype(str).apply(lambda x: re.sub(r'[^\w\s.,!?;:()\-]', '', x, flags=re.UNICODE) if pd.notna(x) else "")
        df['Description'] = df['Description'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

    # FieldAreaSqm - "m" hərfini silir və float64-a keçir
    if 'FieldAreaSqm' in df.columns:
        df['FieldAreaSqm'] = df['FieldAreaSqm'].astype(str).str.replace('m', '', regex=False).str.strip()
        df['FieldAreaSqm'] = pd.to_numeric(df['FieldAreaSqm'], errors='coerce').astype('float64')

    # Title - Müqavilə və ya Çıxarış varsa 1, yoxsa 0
    if 'Title' in df.columns:
        df['Title'] = df['Title'].astype(str).str.lower().apply(lambda x: 1 if any(word in x for word in ['müqavilə', 'çıxarış', 'kupça']) else 0).astype('int64')

    # Repaired - təmir vəziyyətinə görə 0 və ya 1
    if 'Repaired' in df.columns:
        bad_repairs = ['təmirsiz', 'natamam təmirli', 'zəif təmirli']
        good_repairs = ['orta təmirli', 'yaxşı təmirli', 'əla təmirli']
        df['Repaired'] = df['Repaired'].astype(str).str.lower().apply(
            lambda x: 1 if any(good in x for good in good_repairs) else 0
        ).astype('int64')

    if 'Address' in df.columns:
        # Emojiler ve lazimsiz isareler 
        df['Address'] = df['Address'].astype(str).apply(lambda x: re.sub(r'[^\w\s.,\-()]', '', x, flags=re.UNICODE) if pd.notna(x) else "")
        df["Address"] = df["Address"].str.replace("User AgreementOpen in Yandex.Maps", "", regex=False)
        df["Address"] = df["Address"].str.replace("Yandex User AgreementDirections", "", regex=False)
        df["Address"] = df["Address"].str.replace("Ünvan", "", regex=False)
        df["Address"] = df["Address"].apply(lambda x: re.sub(r'\s+', ' ', x).strip())
        df["Address"] = df["Address"].astype("string")

    # SalesPerson - lazımsız hissələri silir
    if 'SalesPerson' in df.columns:
        patterns_to_remove = [r'\(Mülkiyyətçi\)\s*Nömrəni göstər', r'\(Vasitəçi\)\s*Nömrəni göstər', r'Nömrəni göstər']
        for pattern in patterns_to_remove:
            df['SalesPerson'] = df['SalesPerson'].astype(str).str.replace(pattern, '', regex=True, case=False)
        df['SalesPerson'] = df['SalesPerson'].str.strip()

    # SellerType - (Mülkiyyətçi) = 1, (Vasitəçi) = 2
    if 'SellerType' in df.columns:
        df['SellerType'] = df['SellerType'].astype(str).str.lower().apply(
            lambda x: 1 if 'mülkiyyətçi' in x else (2 if 'vasitəçi' in x else 0)
        ).astype('int64')


    # 1. Sütunu əvvəlcə object tipinə çevirək
    df['CategoryID'] = df['CategoryID'].astype(object)

    # 2. Replace tətbiq edək
    df['CategoryID'] = df['CategoryID'].replace({
        "Yeni tikili": 2, 
        "Köhnə tikili": 3, 
        "Həyət evi": 4,
        "Bağ evi": 4,
        "Villa": 4, 
        "Ofis": 5, 
        "Qaraj": 6, 
        "Torpaq": 7, 
        "Obyekt": 8
    })

    # 3. Sonra tam ədədlərə çevir (NULL-lar qalacaq)
    df['CategoryID'] = pd.to_numeric(df['CategoryID'], errors='coerce').astype('Int64')


    df['Contact'] = df['Contact'].str[:15]

    city_map = {
        "Ağcabədi": 17, "Ağdam": 18, "Ağdaş": 19, "Ağstafa": 20, "Ağsu": 21,
        "Astara": 22, "Bakı": 23, "Balakən": 24, "Beyləqan": 25, "Bərdə": 26,
        "Biləsuvar": 27, "Cəlilabad": 28, "Daşkəsən": 29, "Gədəbəy": 30,
        "Gəncə": 31, "Göyçay": 32, "Göygöl": 33, "Hacıqabul": 34, "Xaçmaz": 35,
        "Xankəndi": 36, "Xırdalan": 37, "Xızı": 38, "Xudat": 39, "İmişli": 40,
        "İsmayıllı": 41, "Kürdəmir": 42, "Qax": 43, "Qazax": 44, "Qəbələ": 45,
        "Qobustan": 46, "Quba": 47, "Qusar": 48, "Lerik": 49, "Lənkəran": 50,
        "Masallı": 51, "Mingəçevir": 52, "Naxçıvan": 53, "Naxçıvan MR": 54,
        "Neftçala": 55, "Oğuz": 56, "Saatlı": 57, "Sabirabad": 58, "Salyan": 59,
        "Samux": 60, "Siyəzən": 61, "Sumqayıt": 62, "Şabran": 63, "Şamaxı": 64,
        "Şəki": 65, "Şəmkir": 66, "Şirvan": 67, "Tərtər": 68, "Tovuz": 69,
        "Ucar": 70, "Yardımlı": 71, "Yevlax": 72, "Zaqatala": 73, "Zərdab": 74}




    metro_mapping = {
        "20 yanvar":1, "28 May":2, "8 Noyabr":3, "Avtovağzal":4, "Azadlıq Prospekti":5, "Bakmil":6,
        "Dərnəgül":7, "Elmlər Akademiyası":8, "Əhmədli":9, "Gənclik":10, "Həzi Aslanov":11,
        "İçəri Şəhər":12, "İnşaatçılar":13, "Koroğlu":14, "Memar Əcəmi":15, "Nəriman Nərimanov":16,
        "Nəsimi":17, "Neftçilər":18, "Nizami":19, "Qara Qarayev":20, "Sahil":21,
        "Şah İsmayıl Xətai":22, "Xalqlar Dostluğu":23, "Xocəsən":24, "Ulduz":25, "Cəfər Cabbarlı":26}

    settlement_mapping = {
        "Abşeron r.":1, "Güzdək":2, "Atyalı":3, "Ceyranbatan":4, "Çiçək":5, "Digah":6,
        "Fatmayı":7, "Görədil":8, "Hökməli":9, "Köhnə Corat":10, "Qobu":11, "Masazır":12,
        "Mehdiabad":13, "Məmmədli":14, "Novxanı":15, "Perekeşkül":16, "Saray":17, "Yeni Corat":18,
        "Zuğulba":19, "Binəqədi r.":20, "Alatava 2":21, "28 may":22, "6 mkr":23,
        "7 mkr":24, "8 mkr":25, "9 mkr":26, "Biləcəri":27,
        "Binəqədi":28, "Xocəsən":29, "Xutor":30, "Rəsulzadə":31, "Sulutəpə":32,
        "Xətai r.":33, "Ağ şəhər":34, "Əhmədli":35, "Həzi Aslanov":36, "Köhnə Günəşli":37,
        "NZS":38, "Xəzər r.":39, "Binə":40, "Buzovna":41, "Dübəndi":42, "Gürgən":43,
        "Qala":44, "Mərdəkan":45, "Şağan":46, "Şimal DRES":47, "Şüvəlan":48, "Türkan":49,
        "Zirə":50, "Qaradağ r.":51, "Ələt":52, "Qızıldaş":53, "Qobustan":54, "Lökbatan":55,
        "Müşfiqabad":56, "Puta":57, "Sahil":58, "Səngəçal":59, "Şubanı":60, "Nərimanov r.":61,
        "Böyükşor":62, "Nəsimi r.":63, "1 mkr":64, "2 mkr":65, "3 mkr":66,
        "4 mkr":67, "5 mkr":68, "Kubinka":69, "Nizami r.":70, "8-ci kilometr":71,
        "Keşlə":72, "Pirallahı r.":73, "Sabunçu r.":74, "Albalı":75, "Bakıxanov":76,
        "Balaxanı":77, "Bilgəh":78, "Kürdəxanı":79, "Maştağa":80, "Nardaran":81, "Pirşağı":82,
        "Ramana":83, "Sabunçu":84, "Savalan":85, "Yeni Balaxanı":86, "Yeni Ramana":87, "Zabrat 1":88,
        "Səbail r.":89, "20-ci sahə":90, "Badamdar":91, "Bayıl":92, "Bibiheybət":93, "Şıxov":94,
        "Suraxanı r.":95, "Bahar":96, "Bülbülə":97, "Dədə Qorqud":98, "Əmircan":99, "Günəşli":100,
        "Hövsan":101, "Qaraçuxur":102, "Massiv A":103, "Massiv B":104, "Massiv D":105,
        "Massiv G":106, "Massiv V":107, "Suraxanı":108, "Şərq":109, "Y.Günəşli":110,
        "Y.Suraxanı":111, "Zığ":112, "Yasamal r.":113, "Yasamal":114, "Yeni Yasamal":115, 
        "Dübəndi bağları":116, "Kimya şəhərciyi":117, "Ləhic Bağları":118, "Məhəmmədli":119, 
        "Nasosnu":120, "Nübar":121, "Papanin":122, "Qara şəhər":123, "Şuşa":124, "Ümid":125, 
        "Xaşaxuna":126, "Zabrat 2":127, "Zağulba":128, "6-cı parallel":129, "Alatava 1":130}


    df['CityID'] = df['ListingName'].apply(lambda x: next((city_id for city, city_id in city_map.items() if city in x), 23))

    # Find MetroID using lambda and convert to int64
    df['MetroID'] = df['ListingName'].apply(lambda x: 
        metro_mapping.get(' '.join(x.split(' m.')[0].split()[-2:]), None) if x.endswith(' m.') else None).astype('Int64')

    # Find SettlementID using lambda and convert to int64
    df['SettlementsID'] = df['ListingName'].apply(lambda x: 
        next((settlement_id for settlement, settlement_id in settlement_mapping.items() if settlement in x), None)).astype('Int64')



    df['Currency'] = 'AZN'

    return df

#------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------

def cleaner():
    while True:
        raw = raw_data.get()        
        cleaned = preprocess(raw)        
        cleaned_data.put(cleaned)       
        raw_data.task_done()



def inserter():
    while True:
        cleaned = cleaned_data.get()
        insert_data(cleaned)
        cleaned_data.task_done()

    

#------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------

# Insert Database
def insert_data(df):    
    # Database bağlantısı
    try:
        conn = pyodbc.connect(
            "DRIVER={ODBC Driver 17 for SQL Server};"
            "SERVER=DESKTOP-KIDL0VQ\\SQLEXPRESS;" 
            "DATABASE=RealEstateDB;"
            "Trusted_Connection=yes;")
        cursor = conn.cursor()
        print("Database Connection")
    except Exception as e:
        print(f"Database Connect Xeta: {e}")
        return False

    print(f"Orijinal məlumat sayı: {len(df)}")
    df = df.drop_duplicates(subset=['ListingID'])
    print(f"Dublikatlar silindikdən sonra: {len(df)}")

    # null deyerleri ucun
    def safe_value(value):
        if pd.isna(value):
            return None
        return value

    # datalar database-de yoxlanir
    def is_listing_exists(listing_id, table_name):
        try:
            query = f"SELECT COUNT(*) FROM Dynamic.{table_name} WHERE ListingID = ?"
            cursor.execute(query, (listing_id,))
            result = cursor.fetchone()
            return result[0] > 0
        except Exception as e:
            print(f"Yoxlama xətası {table_name} - {listing_id}: {e}")
            return False

    stats = {
        'apartment': 0,
        'house': 0,
        'office': 0,
        'garage': 0,
        'land': 0,
        'commercial': 0,
        'estate_listing': 0,
        'seller_info': 0,
        'url': 0,
        'labels': 0,
        'description': 0,
        'listing_images': 0 }

    try:        
        estate_query = """
        INSERT INTO Dynamic.EstateListing (ListingID, ListingName, SalesType, DailyRent, CategoryID, SellerType, Price, PricePerSqm, Currency, CityID, Address, Date, Time, MetroID, SettlementsID, Latitude, Longitude)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        
        for index, row in df.iterrows():
            listing_id = safe_value(row["ListingID"])
            
            if not is_listing_exists(listing_id, "EstateListing"):
                try:
                    cursor.execute(estate_query, (listing_id,
                        safe_value(row.get("ListingName")),
                        safe_value(row.get("SalesType")),
                        safe_value(row.get("DailyRent")),
                        safe_value(row.get("CategoryID")),
                        safe_value(row.get("SellerType")),
                        safe_value(row.get("Price")),
                        safe_value(row.get("PricePerSqm")),
                        safe_value(row.get("Currency")),
                        safe_value(row.get("CityID")),
                        safe_value(row.get("Address")),
                        safe_value(row.get("Date")),
                        safe_value(row.get("Time")),
                        safe_value(row.get("MetroID")),
                        safe_value(row.get("SettlementsID")),
                        safe_value(row.get("Latitude")),
                        safe_value(row.get("Longitude"))))
                    stats['estate_listing'] += 1
                except Exception as e:
                    print(f" X EstateListing {listing_id} xətası: {e}")
        conn.commit()


        seller_query = """
        INSERT INTO Dynamic.SellerInfo (ListingID, ListingName, Contact, SellerType, Agency, Residence)
        VALUES (?, ?, ?, ?, ?, ?)
        """
        
        for index, row in df.iterrows():
            listing_id = safe_value(row["ListingID"])
            if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "SellerInfo"):
                try:
                    cursor.execute(seller_query, (
                        listing_id,
                        safe_value(row.get("ListingName")),
                        safe_value(row.get("Contact")),
                        safe_value(row.get("SellerType")),
                        safe_value(row.get("Agency")),
                        safe_value(row.get("Residence"))
                    ))
                    stats['seller_info'] += 1
                except Exception as e:
                    print(f"  ✗ SellerInfo {listing_id} xətası: {e}")

        description_query = """
        INSERT INTO Dynamic.Description (ListingID, Description)
        VALUES (?, ?)
        """
        
        for index, row in df.iterrows():
            listing_id = safe_value(row["ListingID"])
            if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "Description"):
                try:
                    cursor.execute(description_query, (
                        listing_id,
                        safe_value(row.get("Description"))
                    ))
                    stats['description'] += 1
                except Exception as e:
                    print(f"  ✗ Description {listing_id} xətası: {e}")

        conn.commit()


        if df["CategoryID"].isin([1, 2, 3]).any():
            apartment_query = """
            INSERT INTO Dynamic.Apartment (ListingID, CategoryID, FloorNumber, TotalFloors, FieldAreaSqm, CountRoom, Title, Mortgage, Repaired)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            """
            df_apartments = df[df["CategoryID"].isin([1, 2, 3])]
            
            for index, row in df_apartments.iterrows():
                listing_id = safe_value(row["ListingID"])
                if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "Apartment"):
                    try:
                        cursor.execute(apartment_query, (listing_id,
                            safe_value(row["CategoryID"]),
                            safe_value(row.get("FloorNumber")),
                            safe_value(row.get("TotalFloors")),
                            safe_value(row.get("FieldAreaSqm")),
                            safe_value(row.get("CountRoom")),
                            safe_value(row.get("Title")),
                            safe_value(row.get("Mortgage")),
                            safe_value(row.get("Repaired"))))
                        stats['apartment'] += 1
                    except Exception as e:
                        print(f" X Apartment {listing_id} xətası: {e}")

        if (df["CategoryID"] == 4).any():
            house_query = """
            INSERT INTO Dynamic.House (ListingID, CategoryID, FieldAreaSqm, AreaInSot, CountRoom, Mortgage, Repaired)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            """
            df_house = df[df["CategoryID"] == 4]
            
            for index, row in df_house.iterrows():
                listing_id = safe_value(row["ListingID"])
                if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "House"):
                    try:
                        cursor.execute(house_query, (listing_id,
                            safe_value(row["CategoryID"]),
                            safe_value(row.get("FieldAreaSqm")),
                            safe_value(row.get("AreaInSot")),
                            safe_value(row.get("CountRoom")),
                            safe_value(row.get("Mortgage")),
                            safe_value(row.get("Repaired"))))
                        stats['house'] += 1
                    except Exception as e:
                        print(f"X House {listing_id} xətası: {e}")

        if (df["CategoryID"] == 5).any():
            office_query = """
            INSERT INTO Dynamic.Office (ListingID, CategoryID, FieldAreaSqm, [BuildingType (ForOffice)], CountRoom, Title, Repaired)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            """
            df_office = df[df["CategoryID"] == 5]
            
            for index, row in df_office.iterrows():
                listing_id = safe_value(row["ListingID"])
                if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "Office"):
                    try:
                        cursor.execute(office_query, (listing_id,
                            safe_value(row["CategoryID"]),
                            safe_value(row.get("FieldAreaSqm")),
                            safe_value(row.get("BuildingType (ForOffice)")),
                            safe_value(row.get("CountRoom")),
                            safe_value(row.get("Title")),
                            safe_value(row.get("Repaired"))))
                        stats['office'] += 1
                    except Exception as e:
                        print(f" X Office {listing_id} xətası: {e}")

        if (df["CategoryID"] == 6).any():
            garage_query = """
            INSERT INTO Dynamic.Garage (ListingID, CategoryID, FieldAreaSqm, Title)
            VALUES (?, ?, ?, ?)
            """
            df_garage = df[df["CategoryID"] == 6]
            
            for index, row in df_garage.iterrows():
                listing_id = safe_value(row["ListingID"])
                if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "Garage"):
                    try:
                        cursor.execute(garage_query, (listing_id,
                            safe_value(row["CategoryID"]),
                            safe_value(row.get("FieldAreaSqm")),
                            safe_value(row.get("Title"))))
                        stats['garage'] += 1
                    except Exception as e:
                        print(f" X Garage {listing_id} xətası: {e}")


        if (df["CategoryID"] == 7).any():
            land_query = """
            INSERT INTO Dynamic.Land (ListingID, CategoryID, FieldAreaSqm, Title, Mortgage)
            VALUES (?, ?, ?, ?, ?)
            """
            df_land = df[df["CategoryID"] == 7]
            
            for index, row in df_land.iterrows():
                listing_id = safe_value(row["ListingID"])
                if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "Land"):
                    try:
                        cursor.execute(land_query, (listing_id,
                            safe_value(row["CategoryID"]),
                            safe_value(row.get("FieldAreaSqm")),
                            safe_value(row.get("Title")),
                            safe_value(row.get("Mortgage")))) 
                        stats['land'] += 1
                    except Exception as e:
                        print(f"X Land {listing_id} xətası: {e}")

        if (df["CategoryID"] == 8).any():
            commercial_query = """
            INSERT INTO Dynamic.CommercialProperty (ListingID, CategoryID, FieldAreaSqm, Title, Mortgage, Repaired)
            VALUES (?, ?, ?, ?, ?, ?)
            """
            df_commercial = df[df["CategoryID"] == 8]
            
            for index, row in df_commercial.iterrows():
                listing_id = safe_value(row["ListingID"])
                if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "CommercialProperty"):
                    try:
                        cursor.execute(commercial_query, (listing_id,
                            safe_value(row["CategoryID"]),
                            safe_value(row.get("FieldAreaSqm")),
                            safe_value(row.get("Title")),
                            safe_value(row.get("Mortgage")),
                            safe_value(row.get("Repaired"))))
                        stats['commercial'] += 1
                    except Exception as e:
                        print(f"X Commercial Property {listing_id} xətası: {e}")

        conn.commit()


        # URL
        url_query = """
        INSERT INTO Dynamic.URL (ListingID, URL)
        VALUES (?, ?)
        """
        
        for index, row in df.iterrows():
            listing_id = safe_value(row["ListingID"])
            if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "URL"):
                try:
                    cursor.execute(url_query, (
                        listing_id,
                        safe_value(row.get("URL"))
                    ))
                    stats['url'] += 1
                except Exception as e:
                    print(f" X URL {listing_id} xətası: {e}")


        labels_query = """
        INSERT INTO Dynamic.Labels (ListingID, Labels)
        VALUES (?, ?)
        """

        def is_listing_label_exists(listing_id, label):
            check_query = "SELECT 1 FROM Dynamic.Labels WHERE ListingID = ? AND Labels = ?"
            cursor.execute(check_query, (listing_id, label))
            return cursor.fetchone() is not None

        for index, row in df.iterrows():
            listing_id = safe_value(row["ListingID"])
            if is_listing_exists(listing_id, "EstateListing"):
                labels_raw = row.get("Labels", "")
                
                labels_list = []
                if labels_raw:
                    if isinstance(labels_raw, str):
                        clean_str = str(labels_raw).replace("[", "").replace("]", "").replace("'", "").replace('"', "")
                        labels_list = [x.strip() for x in clean_str.split(",") if x.strip()]
                    elif isinstance(labels_raw, list):
                        labels_list = [str(x).strip() for x in labels_raw if str(x).strip()]
                
                for label in labels_list:
                    if not is_listing_label_exists(listing_id, label):
                        try:
                            cursor.execute(labels_query, (listing_id, label))
                            stats['labels'] += 1
                        except Exception as e:
                            print(f" X Labels {listing_id} ({label}) xətası: {e}")


        def is_image_exists(image_url):
            try:
                check_query = "SELECT COUNT(*) FROM Dynamic.ListingImages WHERE ImageURL = ?"
                cursor.execute(check_query, (image_url,))
                result = cursor.fetchone()
                return result[0] > 0
            except Exception as e:
                print(f"Image yoxlama xətası: {e}")
                return False

        listing_images_query = """
        INSERT INTO Dynamic.ListingImages (ListingID, ImageURL)
        VALUES (?, ?)
        """

        processed_images = set()

        for index, row in df.iterrows():
            listing_id = safe_value(row["ListingID"])
            if is_listing_exists(listing_id, "EstateListing"):
                image_links = row.get("Images")
                
                if image_links and listing_id:
                    if isinstance(image_links, str):
                        try:
                            import json
                            image_list = json.loads(image_links)
                        except:
                            image_list = [link.strip() for link in image_links.split(',') if link.strip()]
                    elif isinstance(image_links, list):
                        image_list = image_links
                    else:
                        image_list = []
                    
                    for image_url in image_list:
                        image_url_clean = safe_value(image_url)
                        
                        if image_url_clean and image_url_clean not in processed_images:
                            if not is_image_exists(image_url_clean):
                                try:
                                    cursor.execute(listing_images_query, (
                                        listing_id,
                                        image_url_clean
                                    ))
                                    stats['listing_images'] += 1
                                    processed_images.add(image_url_clean)
                                except Exception as e:
                                    print(f"X Image {listing_id} xətası: {e}")

        # Final commit
        conn.commit()
        print("Uğurlu")

    except Exception as e:
        print(f"Ümumi xəta baş verdi: {e}")
        try:
            conn.rollback()
            print("Rollback edildi")
        except:
            pass
        return False

    finally:
        try:
            cursor.close()
            conn.close()
            print("Database bağlantısı bağlandı")
        except:
            pass

    # Nəticələr
    print(f"   EstateListing: {stats['estate_listing']}")
    print(f"   SellerInfo:    {stats['seller_info']}")
    print(f"   Description:   {stats['description']}")
    print(f"   Apartment:     {stats['apartment']}")
    print(f"   House:         {stats['house']}")
    print(f"   Office:        {stats['office']}")
    print(f"   Garage:        {stats['garage']}")
    print(f"   Land:          {stats['land']}")
    print(f"   Commercial:    {stats['commercial']}")
    print(f"   URL:           {stats['url']}")
    print(f"   Labels:        {stats['labels']}")
    print(f"   Images:        {stats['listing_images']}")
    print(f"   TOPLAM:        {sum(stats.values())}")

    return True




if __name__ == "__main__":
    Thread(target=scrap, daemon=True).start()
    Thread(target=cleaner, daemon=True).start()
    Thread(target=inserter, daemon=True).start()

    # Prosesi aciq saxlamaq ucun
    raw_data.join()
    cleaned_data.join()
